# Cohort Analysis
### of Resellers in a Dropshipping Company

In this cohort analysis, I measured the retention rate of group of resellers who place at least 9 order in their first month of activation and place 9+ orders in the subsequent weeks. As a reseller makes an order in week n, she is considered as retained for week n, week n+1, week n+2, and week n+3. For this report, day 0 is the day in which resellers make their 9th order. So, here is my approach to create this report;

1. Find resellers with 9+ orders in their first month.
2. Among resellers from the previous step, find those who place 9+ orders in the subsequent weeks.
3. For week n, retention = number of resellers with 9+ orders in [week n-3, week n] interval.
4. Divide the retention by the number of resellers with 9+ orders in week 1.

with base1 AS (SELECT
    `orders`.`ID` AS `OrderID`,
    `orders`.`UserID` AS `UserID`,
    `orders`.`CreatedAt` AS `OrderDate`,
    `orders`.`FakeDelete` AS `FakeDelete`,
    `Users`.`TotalOrders` AS `TotalOrders`,
    `Users`.`TotalGrossOrders` AS `GrossOrders`,
    `Users`.`CreatedAt` AS `JoinDate`,
    `Orders Is Net`.`isGross` AS `isGross`,
    `Users First Order`.`FirstOrderAt` AS `ActivationDate`,
    -- DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`) + 1 AS `Day`,
    CEILING((DATEDIFF(`orders`.`CreatedAt`, `Users First Order`.`FirstOrderAt`)+1)/7) as `WeekActivation`,
    CEILING((DATEDIFF(`orders`.`CreatedAt`, `Users First Order`.`FirstOrderAt`)+1)/30) as `MonthActivation`,
    ROW_NUMBER() OVER (PARTITION BY UserID ORDER BY orders.CreatedAt ASC ) AS `OrderOrder`
    FROM
    `orders`
    INNER JOIN `users` AS `Users` ON `orders`.`UserID` = `Users`.`ID`
    INNER JOIN `vw_orders_is_net` AS `Orders Is Net` ON `orders`.`ID` = `Orders Is Net`.`OrderID`
    INNER JOIN `vw_users_first_order` AS `Users First Order` ON `Users`.`ID` = `Users First Order`.`UserID`
    WHERE
    (`Orders Is Net`.`isGross` = 1)
    AND (`orders`.`FakeDelete` = 0)
    ORDER BY `orders`.`UserID`),
    
-- Users with 9+ orders in f30D. GraduationDate=date of 9th order
with9Orders AS (SELECT UserID, OrderDate AS GraduationDate FROM base1
    WHERE OrderOrder = 9 AND MonthActivation = 1),
    
-- Calculate GraduationWeek i.e. week1 = first week after nineth order
graduation AS (SELECT base1.UserID, base1.OrderID, CEILING((DATEDIFF(base1.OrderDate, with9Orders.graduationDate)+1)/7) AS WeekGraduation
    FROM base1
    INNER JOIN with9Orders ON base1.UserID = with9Orders.UserID
    WHERE OrderDate >= graduationDate),
    
-- Select weeks with 9+ Orders
weekWith9 AS (SELECT UserID, WeekGraduation, COUNT(OrderID) AS orders
    FROM graduation
    GROUP BY UserID, WeekGraduation
    HAVING orders > 8),
    
-- Create table of weeks
myWeeks AS (SELECT DISTINCT WeekGraduation AS Week 
    FROM weekWith9 
    ORDER BY WeekGraduation),

-- Join tables to calculate retention    
myJoin AS (SELECT myWeeks.Week AS WWeek, weekWith9.WeekGraduation as TWeek, UserID
    FROM myWeeks, weekWith9
    WHERE weekWith9.WeekGraduation BETWEEN (myWeeks.Week-(CASE WHEN MOD(myWeeks.Week, 4)> 0 THEN MOD(myWeeks.Week, 4)-1 ELSE 3 END)) AND myWeeks.Week),

myFinal AS (SELECT WWeek AS Week, COUNT(DISTINCT TWeek) AS Duration, MIN(TWeek) AS StartWeek, MAX(TWeek) AS EndWeek, COUNT(DISTINCT UserID) AS ActivesInInterval
    FROM  myJoin
    GROUP BY Week),

allActivated AS (SELECT COUNT(DISTINCT UserID) AS AllActivation 
    FROM with9Orders)

SELECT Week, StartWeek, Duration, EndWeek, ActivesInInterval, (ActivesInInterval/AllActivation)*100 AS PercentActivesInInterval
    FROM  myFinal, allActivated
    WHERE Week > 4;
    

